## 증권리포트 크롤링

- 매일경제증권센터 증권리포트 항목 크롤링
- 검색어, 검색할 maxPage 설정

In [52]:
import requests
from bs4 import BeautifulSoup
from urllib import parse

#증권리포트 제목, 내용, url을 분리, 저장하는 함수
def get_report_inf(url):
    headers = {'Accept-Encoding': 'deflate'}
    res = requests.get(url=url, headers=headers)
    # BeautifulSoup을 이용해 웹 html을 파싱
    soup = BeautifulSoup(res.content, 'html.parser')#, from_encoding="euc-kr"
    title = soup.find('div', {'id':'Titless'}).text
    text = soup.find('div', {'id':'Conts'}).text
    report_data = {}
    report_data['title'] = title.strip()  # 기사 제목
    text = text.replace("\r", " ").strip()
    report_data['text'] = text.replace("<br>", " ").strip()  # 기사 내용에서 줄바꿈을 지우고, 좌우 공백 제거
    report_data['url'] = url  # 기사 링크
    
    return report_data

In [53]:
#증권리포트 크롤링 data를 반환하는 함수

def get_report_list(query,  # 검색어
                    page=1  # 뉴스 리스트 페이지
                      ):
    
    
    #url='http://vip.mk.co.kr/newSt/news/news_list2.php?sCode=110&p_page={page}&search=' + parse.quote(query, encoding='euc-kr')
    url = 'http://vip.mk.co.kr/newSt/news/news_list.php?sCode=21'
    #res = requests.get(url.format(page=page))    
    res = requests.get(url)
    
    # BeautifulSoup을 이용해 웹 html을 파싱
    soup = BeautifulSoup(res.content, 'html.parser')
    #print(soup)
    
    # html 구조에서 해당 기사로 접근할 수 있는 url을 추출    
    table = soup.find('table', {'class':'table_6'})
    alinks = table.findAll('a')
    
    report_list = []
    
    for alink in alinks:
        #alink['href'] = alink['href'].replace("./", "http://vip.mk.co.kr/newSt/news/")
        report_url = alink['href']
        report_list.append(get_report_inf(report_url))
    
    return report_list

In [54]:
#증권리포트 크롤링 실행
articles = []
searchword = '[반도체]'
#for num in range(1, 4):
    #articles.extend(get_report_list(searchword, page=num))
articles = get_report_list(searchword)
print("Total articles: {:,}".format(len(articles)))
print("="*20, "Article sample", "="*20)
print(articles[0])

Total articles: 50
==================== Article sample ====================
{'title': '외인 매도에도 코스피 1,960대 강세…코스닥은 1%대 상승', 'text': '코스피 소폭 상승  (PG)     코스피가 21일 사흘째 상승 마감했다.     이날 코스피는 전 거래일보다 4.40포인트(0.22%) 오른 1,964.65에 종료했다.     지수는 전장보다 5.68포인트(0.29%) 내린 1,954.57로 출발했으나 장중 한때 1,971.12까지 오르는 등 대체로 강세 흐름을 보였다.     유가증권시장에서는 개인이 1천19억원, 기관이 56억원어치를 각각 순매수했다.googletag.display(\'google_dfp_MC_2x1,fluid\');외국인은 1천294억원을 순매도했다.     서상영 키움증권 연구원은 "미중 무역협상에 대한 기대가 여전한 가운데 장중 원/달러 환율이 달러당 1,200원 아래로 내려오는 등 외환시장이 안정된 흐름을 보이면서 증시에 호재로 작용했다"며 "다만 미 연방공개시장위원회(FOMC) 의사록 공개를 앞두고 관망 심리가 우세하다 보니 상승 폭은 제한적이었다"고 설명했다.     시가총액 상위 종목 중에서는 삼성전자[005930](0.11%), 네이버[035420](1.37%), 현대모비스[012330](0.20%), LG화학[051910](1.23%), 셀트리온[068270](0.97%), SK텔레콤[017670](1.71%) 등이 올랐다.     SK하이닉스[000660](-0.79%), 현대차[005380](-0.79%), 신한지주[055550](-0.62%), 포스코[005490](-0.47%) 등은 내렸다.     업종별로는 의료정밀(2.14%), 통신(1.52%), 건설(1.06%), 의약품(0.97%), 증권(0.49%) 등이 강세였고 섬유·의복(-0.86%), 종이·목재(-0.15%), 전기·전자(-0.10%)는 약세를 보였다.      프로그

## Text preprocessing

- 형태소 분류작업 (정규화, 어근화작업도 수행)

In [55]:
from konlpy.tag import Okt
from collections import Counter
# https://konlpy-ko.readthedocs.io/ko/v0.4.3/morph/
tagger = Okt()  # Twitter 형태소 분석기 (Open Korean Text)


#키워드 추출 조건 설정
#input - 키워드로 포함하고자하는 품사 입력, 키워드로 포함하지 않을 금지단어 입력
#output - 기사갯수 100단위로 출력, 시data 첫번째 기사항목 출력
words_list = []
vocab = Counter() # 글자를 숫자로 만들기 위함
tag_set = set(['Noun', 'Alpha'])  # 키워드로 포함하고자 하는 품사 (명사, 알파벳)
stopwords = set([])  # 키워드로 포함하지 않을 금지 단어 -> 하면서 포함되도록..
stopwords.add(searchword)  #검색어 자체가 키워드에 포함되지 않게한다


for i, article in enumerate(articles):
        
    words = tagger.pos(article['text'], norm=True, stem=True) #정규화, 어근화도 진행
    tmp_words = []

    for w, t in words:
        if t in tag_set and len(w) > 1 and w not in stopwords:
            tmp_words.append(w)
    vocab.update(tmp_words)
    words_list.append((tmp_words, article))
    


vocab = sorted([w for w, freq in vocab.most_common(10000)]) # voca size
word2id = {w: i for i, w in enumerate(vocab)}  # {"SK": 0, "로봇": 1, "딥러닝": 2, ...}

new_words_list = []
for words, article in words_list:
    words = [w for w in words if w in word2id]  # vocab에 있는 단어만 사용
    # 기사 내 서로 다른 단어 개수가 5개 이상인 기사만 사용    
    if len(words) > 5:
        new_words_list.append((words, article))

## Build term-document matrix

In [56]:
#input - words_list, vocab
from sklearn.feature_extraction.text import TfidfTransformer
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import pandas as pd


print(len(words_list))
print(len(vocab))
term_doc_matrix = np.zeros((len(words_list), len(vocab)), dtype=np.float32) # 메트릭스 만듬
for i, (words, article) in enumerate(new_words_list):
    for word in words:
        term_doc_matrix[i, word2id[word]] += 1
        
term_doc_matrix = TfidfTransformer().fit_transform(term_doc_matrix) #써이키런 -> 온갖 라이버러리 다 있다
print("Term document matrix shape:", term_doc_matrix.shape)
print(term_doc_matrix)

50
2454
Term document matrix shape: (50, 2454)
  (0, 2449)	0.1253440141065363
  (0, 2443)	0.01667349776561923
  (0, 2436)	0.01667349776561923
  (0, 2421)	0.1253440141065363
  (0, 2410)	0.05432014531565755
  (0, 2394)	0.06267200705326816
  (0, 2386)	0.05432014531565755
  (0, 2378)	0.05432014531565755
  (0, 2377)	0.0447099912633571
  (0, 2373)	0.05796848279814595
  (0, 2313)	0.048818914142988
  (0, 2310)	0.01667349776561923
  (0, 2307)	0.01667349776561923
  (0, 2287)	0.05432014531565755
  (0, 2265)	0.2506880282130726
  (0, 2263)	0.06930125282066259
  (0, 2188)	0.048818914142988
  (0, 2176)	0.05796848279814595
  (0, 2156)	0.01667349776561923
  (0, 2132)	0.15401771109225457
  (0, 2131)	0.1159369655962919
  (0, 2130)	0.048818914142988
  (0, 2124)	0.06267200705326816
  (0, 2077)	0.05796848279814595
  (0, 1996)	0.06267200705326816
  :	:
  (48, 148)	0.042317513521811824
  (48, 147)	0.1692700540872473
  (48, 146)	0.042317513521811824
  (48, 144)	0.042317513521811824
  (48, 143)	0.13029952609752

## Topic modeling

In [57]:
# https://scikit-learn.org/stable/modules/classes.html#module-sklearn.decomposition
from sklearn.decomposition import NMF
 
K = 5  # topic 개수지정
nmf = NMF(n_components=K, alpha=0.1)  # Non-negative matrix factorization
W = nmf.fit_transform(term_doc_matrix) # 토픽모델링 해준다.
H = nmf.components_

print("W shape:", W.shape)
print("H shape:", H.shape)

W shape: (50, 5)
H shape: (5, 2454)
